In [75]:
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
import sklearn


sample_submission.csv - A submission file in the correct format.
PassengerId - Id for each passenger in the test set.
Transported - The target. For each passenger, predict either True or False.


In [97]:
sample_submission_df = pl.read_csv('sample_submission.csv')
train_df = pl.read_parquet('train_df.parquet')
test_df = pl.read_parquet('test_df.parquet')

In [98]:
train_df = train_df.drop_nulls()
y = train_df['Transported']
X = train_df.drop('Transported')

y_test = test_df['Transported']
X_test = test_df.drop('Transported')

In [100]:
classifier = sklearn.ensemble.HistGradientBoostingClassifier(
    learning_rate=0.1,
    max_iter=2000,
    max_leaf_nodes=None,
    max_depth=None,
    min_samples_leaf=32,
    l2_regularization=1.0,
    max_features=0.7,
    categorical_features="from_dtype",
    warm_start=True,
    scoring="f1",
    validation_fraction=0.3,
    verbose=2,
    random_state=42,
)

In [101]:
classifier.fit(X, y)
score = classifier.score(X, y)
print('Score:', score)

Binning 0.001 GB of training data: 0.020 s
Fitting gradient boosted rounds:
[1/2000] 1 tree, 116 leaves, max depth = 14, in 0.019s
[2/2000] 1 tree, 121 leaves, max depth = 16, in 0.020s
[3/2000] 1 tree, 116 leaves, max depth = 17, in 0.019s
[4/2000] 1 tree, 118 leaves, max depth = 13, in 0.020s
[5/2000] 1 tree, 134 leaves, max depth = 16, in 0.022s
[6/2000] 1 tree, 135 leaves, max depth = 20, in 0.022s
[7/2000] 1 tree, 125 leaves, max depth = 14, in 0.022s
[8/2000] 1 tree, 135 leaves, max depth = 19, in 0.023s
[9/2000] 1 tree, 145 leaves, max depth = 14, in 0.022s
[10/2000] 1 tree, 131 leaves, max depth = 14, in 0.021s
[11/2000] 1 tree, 138 leaves, max depth = 13, in 0.022s
[12/2000] 1 tree, 142 leaves, max depth = 17, in 0.031s
[13/2000] 1 tree, 137 leaves, max depth = 16, in 0.024s
[14/2000] 1 tree, 144 leaves, max depth = 16, in 0.023s
[15/2000] 1 tree, 151 leaves, max depth = 21, in 0.025s
[16/2000] 1 tree, 140 leaves, max depth = 17, in 0.023s
[17/2000] 1 tree, 147 leaves, max dep

In [104]:
pred_proba = classifier.predict_proba(X_test)
preds = classifier.predict(X_test)


In [105]:
test_csv = pl.read_csv('test.csv')
transported = pl.Series('Transported', preds).cast(pl.String).str.to_titlecase()
submission_df = test_csv.select('PassengerId').hstack(pl.DataFrame(transported))
submission_df.write_csv('spaceship-titanic-submission.csv')


In [107]:
!kaggle competitions submit -c spaceship-titanic -f spaceship-titanic-submission.csv -m "spaceship-titanic-submission.csv"